In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 4))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 4))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [2]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

garbageCollectionDate[20]

'ballarat-garbage-collection-zones.json'

In [3]:
readInJson(garbageCollectionDate[20]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_4a7a3e5b_a20c_431e_bd21_ada983b0566f.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'collection': 'Friday Collection 1', 'waste':..."
1,Feature,ckan_4a7a3e5b_a20c_431e_bd21_ada983b0566f.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'collection': 'Friday Collection 2', 'waste':..."


In [4]:
df = readInJson(garbageCollectionDate[20])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 4))
df['long'] = df.geo.apply(lambda x: round(x[1], 4))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [5]:
newCols = ['landfill_colDay', 
           'recyclable_colDay']

newColsKeys = ['waste', 
               'recycle']

for key, val in dict(zip(newCols, newColsKeys)).items(): 
    df[key] = df.properties.apply(lambda x: x[val])
    
for i in ['landfill', 'recyclable']: 
    df[f'{i}_date'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[1])
    df[f'{i}_date'] = pd.to_datetime(df[f'{i}_date'], format='%Y-%m-%d').dt.day_name()
    df[f'{i}_freq'] = df[f'{i}_colDay'].apply(lambda x: x.split('/')[-1])
    df[f'{i}_freq'] = df[f'{i}_freq'].apply(lambda x: 'Every Second ' if x == 'P2W' else 'Every ')
    
    df[f'{i}_colDay'] = df[f'{i}_freq'] + df[f'{i}_date']
    
    df.drop([f'{i}_freq', f'{i}_date'], axis = 1, inplace = True)

In [6]:
ballarat = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'landfill_colDay', 'recyclable_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'landfill_colDay', 'recyclable_colDay'], 
                           ['landfill', 'landfill', 'recyclable'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

ballarat.to_csv('../GBD_ballarat.csv', index = False)